In [8]:
import pyfixest as pf

import polars as pl
import pandas as pd

import numpy as np

In [9]:
df_castle = pd.read_stata("https://github.com/scunning1975/mixtape/raw/master/castle.dta")
df_castle = df_castle.astype({"post": bool, "popwt": np.float64})

df_untreated = df_castle.loc[df_castle["post"] == 0, :]

fit_untrt = pf.feols("l_homicide ~ 1 | state + year", data=df_untreated)
fit_untrt_wgt = pf.feols("l_homicide ~ 1 | state + year", data=df_untreated, weights="popwt")

yhat = fit_untrt.predict(newdata=df_castle)
yhat_wgt = fit_untrt_wgt.predict(newdata=df_castle)

# Getting first stage predictions

yhat_nn = fit_untrt.predict()
yhat_wgt_nn = fit_untrt_wgt.predict()


In [10]:
fit_did2s_unwgt = pf.did2s(
    data=df_castle,
    yname="l_homicide",
    first_stage="~ 0 | sid + year",
    second_stage="~ i(post, ref=0)",
    treatment="post",
    cluster="state",
)

fit_did2s_unwgt.summary()

###

Estimation:  DID2S
Dep. var.: l_homicide_hat, Fixed effects: 0
Inference:  CRV1 (GMM)
Observations:  550

| Coefficient                              |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:-----------------------------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| C(post, contr.treatment(base=0))[T.True] |      0.067 |        0.057 |     1.173 |      0.246 | -0.048 |   0.181 |
---
RMSE: 0.183 R2: 0.015 


In [11]:
fit_did2s_wgt = pf.did2s(
    data=df_castle,
    yname="l_homicide",
    first_stage="~ 0 | sid + year",
    second_stage="~ i(post, ref=0)",
    treatment="post",
    cluster="state",
    weights="popwt"
)

fit_did2s_wgt.summary()

###

Estimation:  DID2S
Dep. var.: l_homicide_hat, Fixed effects: 0
Inference:  CRV1 (GMM)
Observations:  550

| Coefficient                              |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:-----------------------------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| C(post, contr.treatment(base=0))[T.True] |      0.075 |        0.035 |     2.124 |      0.039 |  0.004 |   0.146 |
---



In [12]:
fit_did2s_wgt.coef()

Coefficient
C(post, contr.treatment(base=0))[T.True]    0.075141
Name: Estimate, dtype: float64

In [14]:
df_castle.dtypes

state           object
year             int16
sid               int8
cdl            float32
pre2_cdl       float32
                ...   
_Iyear_2007       int8
_Iyear_2008       int8
_Iyear_2009       int8
_Iyear_2010       int8
popwt          float64
Length: 185, dtype: object